<a href="https://colab.research.google.com/github/Mkhan2317/Market_Analytics_Web_App/blob/main/Stock_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit
!pip install pyngrok

In [ ]:
# Installing Ta-Lib Library
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib


In [32]:
%%writefile app.py

import streamlit as st
import pandas as pd
import talib
import yfinance as yf
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

def get_sp500_components():
    # Retrieve S&P 500 components from Wikipedia
    df = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
    tickers = df["Symbol"].to_list()
    tickers_companies_dict = df.set_index("Symbol")["Security"].to_dict()
    return tickers, tickers_companies_dict

indicators = [
    "Simple Moving Average",
    "Exponential Moving Average",
    "Relative Strength Index",
]

def apply_indicator(indicator, data):
    # Apply the selected technical analysis indicator to the data
    if indicator == "Simple Moving Average":
        sma = talib.SMA(data["Close"])
        return pd.DataFrame({"Close": data["Close"], "SMA": sma})
    elif indicator == "Exponential Moving Average":
        ema = talib.EMA(data["Close"])
        return pd.DataFrame({"Close": data["Close"], "EMA": ema})
    elif indicator == "Relative Strength Index":
        rsi = talib.RSI(data["Close"])
        return pd.DataFrame({"Close": data["Close"], "RSI": rsi})

st.title("Stock Data Analysis")
st.write("A simple app to download stock data and apply technical analysis indicators.")

st.sidebar.header("Stock Parameters")

# Retrieve S&P 500 tickers and company names
available_tickers, tickers_companies_dict = get_sp500_components()

# User selects a ticker from the sidebar
ticker = st.sidebar.selectbox(
    "Ticker", available_tickers, format_func=tickers_companies_dict.get
)

# User selects a date range from the sidebar
start = st.sidebar.date_input("Start date:", pd.Timestamp("2020-01-01"))
end = st.sidebar.date_input("End date:", pd.Timestamp("2023-12-31"))

# Download stock data for the selected ticker and date range
data = yf.download(ticker, start, end)

# User selects a technical analysis indicator
selected_indicator = st.selectbox("Select a technical analysis indicator:", indicators)

# Display the selected technical analysis indicator and its chart
if not data.empty:
    indicator_data = apply_indicator(selected_indicator, data)
    st.write(f"{selected_indicator} for {ticker}")
    st.line_chart(indicator_data)

    st.write("Stock data for", ticker)
    st.dataframe(data)
else:
    st.write("No data available for the selected date range and ticker.")


Overwriting app.py


In [33]:
!ngrok authtoken 2lPK0JW1gGHKc1obVsOnbcp83mu_6yLc7PRqJj5Qp3XHue4sX


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [34]:
from pyngrok import ngrok

# Terminate open tunnels if running
ngrok.kill()

# Start the Streamlit app
!streamlit run app.py &>/dev/null&

# Open a tunnel on the default port 8501 using the correct syntax
public_url = ngrok.connect(addr="8501", bind_tls=True)  # Specify the address as a string
print(f"Streamlit app is running on: {public_url}")

Streamlit app is running on: NgrokTunnel: "https://d061-35-239-107-160.ngrok-free.app" -> "http://localhost:8501"
